In [2]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_cube_files import CUBE

# dsgdb9nsd_002626

In [3]:
ve38 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube')
ve30 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube')
ve23 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube')
ve15 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube')
ve8 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube')
ve0 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube')

In [18]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from alchemy_tools import atomic_energy_decomposition
p38 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube'
p30 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube'
p23 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube'
p15 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube'
p8 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube'
p0 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube'
cubes = [(p0, 0), (p8, 8/38), (p15, 15/38), (p23, 23/38), (p30, 30/38), (p38, 1)]
# ae, alchpots = atomic_energy_decomposition(cubes)
atomic_energy_decomposition(cubes, '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/output')

In [4]:
print(ae)

[-52.41941963 -38.81593713 -39.49926896 -39.88880231 -38.57914187
 -36.29823107 -32.4020716   -5.39594217  -5.70814393  -5.64600143
  -5.78079931  -5.79580329  -5.5379296   -5.53118386  -4.54357057]


# Integration

In [5]:
from alchemy_tools import integrate_lambda_density, calculate_atomic_energies, calculate_alchemical_potential

In [6]:
lambda_densities = [ve0, ve8, ve15, ve23, ve30, ve38]
density_arrays = []
for dens in lambda_densities:
    dens.scale()
    density_arrays.append(dens.data_scaled)
lam_vals = [0, 8/38, 15/38, 23/38, 30/38, 1]

In [7]:
av_dens = integrate_lambda_density(density_arrays, lam_vals)

# Plots

In [8]:
# projection on line
x = np.linspace(0, 20, 175)
sa = (1,2)
av_dens_1D = av_dens.sum(axis=sa)
pd_1D = [ve38.project(sa), ve30.project(sa), ve23.project(sa), ve15.project(sa), ve8.project(sa), ve0.project(sa), av_dens_1D]
label = [r'$\rho(\lambda = 1.0)$',r'$\rho(\lambda \approx 0.79)$', r'$\rho(\lambda \approx 0.61)$', r'$\rho(\lambda \approx 0.39)$', r'$\rho(\lambda \approx 0.21)$', r'$\rho(\lambda = 0)$', r'$\tilde{\rho} \approx \int_0^1 d\lambda \rho(\lambda, r)$']
fig, ax = plt.subplots(1,1)
for idx, pd in enumerate(pd_1D):
    # bigger linewidth for lambda-averaged density
    if idx == len(pd_1D)-1:
        ax.plot(x, pd, '--', label=label[idx])
    else:
        ax.plot(x, pd, label=label[idx])

ax.set_title('Boxsize = 20 Ang')
ax.set_xlabel('Cell coordinate (Ang)')
ax.set_xlim(0.5, 19.5)
ax.set_ylabel(r'Projected Density $\rho(x_0)$ (Bohr$^{-3}$) ')

ax.legend()

# Alchemical Potential and Atomic Energies

In [9]:
nuclei = ve38.atoms
meshgrid = ve38.get_grid()
meshgrid_xyz = np.vstack([_.flatten() for _ in meshgrid]).T

In [10]:
nuc=np.array(ve38.atoms)[0][1:].T

In [11]:
np.array(ve38.atoms)[0][1:4]

array([14.673877, 14.892382, 22.556505])

In [12]:
calculate_alchemical_potential(av_dens, meshgrid, nuc)

-6.552427453817083

In [13]:
# %timeit atomic_energies = get_atomic_energies(av_dens, nuclei, meshgrid)
atomic_energies = calculate_atomic_energies(av_dens, nuclei, meshgrid)

In [14]:
# %timeit dist_gpt_nuc = np.linalg.norm(meshgrid_xyz - nuclei[0][1:], axis=1)
#(density.flatten()/dist_gpt_nuc).sum()


In [15]:
atomic_energies

(array([-52.41941963, -38.81593713, -39.49926896, -39.88880231,
        -38.57914187, -36.29823107, -32.4020716 ,  -5.39594217,
         -5.70814393,  -5.64600143,  -5.78079931,  -5.79580329,
         -5.5379296 ,  -5.53118386,  -4.54357057]),
 array([-6.55242745, -6.46932286, -6.58321149, -6.64813372, -6.42985698,
        -6.04970518, -5.40034527, -5.39594217, -5.70814393, -5.64600143,
        -5.78079931, -5.79580329, -5.5379296 , -5.53118386, -4.54357057]))

# Atomisation Energies

In [1]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from alchemy_tools import atomic_energy_decomposition
p38 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube'
p30 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube'
p23 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube'
p15 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube'
p8 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube'
p0 = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube'
cubes = [(p0, 0), (p8, 8/38), (p15, 15/38), (p23, 23/38), (p30, 30/38), (p38, 1)]
# cubes = [(p0, 0)]
# atomic energies and alchemical potentials
nuclei, ae, ap = atomic_energy_decomposition(cubes, '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/output')

In [2]:
print(nuclei, '\n', ae, '\n', ap)

[[ 8.       14.673877 14.892382 22.556505]
 [ 6.       15.987601 15.23968  20.733272]
 [ 6.       18.417982 16.742994 20.722894]
 [ 6.       18.355279 18.881106 18.757507]
 [ 6.       20.904635 20.281293 18.574689]
 [ 6.       20.870138 22.316401 16.719434]
 [ 6.       20.787734 23.981867 15.177496]
 [ 1.       15.461396 14.442354 18.85286 ]
 [ 1.       18.756163 17.463877 22.630985]
 [ 1.       19.958001 15.428753 20.248224]
 [ 1.       17.872555 18.116434 16.893851]
 [ 1.       16.869849 20.238529 19.241177]
 [ 1.       21.40558  21.047592 20.43611 ]
 [ 1.       22.399672 18.92563  18.092346]
 [ 1.       20.738454 25.460024 13.821571]] 
 [-52.41941963 -38.81593713 -39.49926896 -39.88880231 -38.57914187
 -36.29823107 -32.4020716   -5.39594217  -5.70814393  -5.64600143
  -5.78079931  -5.79580329  -5.5379296   -5.53118386  -4.54357057] 
 [-6.55242745 -6.46932286 -6.58321149 -6.64813372 -6.42985698 -6.04970518
 -5.40034527 -5.39594217 -5.70814393 -5.64600143 -5.78079931 -5.79580329
 -5.5

In [4]:
from explore_qml_data import get_property, get_free_atom_data
from alchemy_tools import get_free_atom_energies
# total energy from B3LYP
total_energy = get_property('/home/misa/datasets/qm9/dsgdb9nsd_002626.xyz', 'U0')
# energies of the free atoms in qm9
free_atoms = get_free_atom_data()
# free atom energy for every atom in compound
free_en = get_free_atom_energies(nuclei[:,0], free_atoms)

In [5]:
print(total_energy, '\n', free_atoms, '\n', free_en)

-308.496367 
 {'H': -0.500273, 'C': -37.846772, 'N': -54.583861, 'O': -75.064579, 'F': -99.71873} 
 [-75.064579 -37.846772 -37.846772 -37.846772 -37.846772 -37.846772
 -37.846772  -0.500273  -0.500273  -0.500273  -0.500273  -0.500273
  -0.500273  -0.500273  -0.500273]


In [6]:
from alchemy_tools import calculate_atomisation_energies 
# calculate atomisation energies
at=calculate_atomisation_energies(ae, total_energy, free_en)
at

array([23.53488468, -0.07943982, -0.76277164, -1.152305  ,  0.15735545,
        2.43826624,  6.33442572, -4.00594386, -4.31814562, -4.25600311,
       -4.39080099, -4.40580498, -4.14793128, -4.14118554, -3.15357225])

In [7]:
print('Atomisation energy decomposition = %f' %calculate_atomisation_energies(ae, total_energy, free_en).sum())
print('Atomisation energy qm9 = %f ' % (total_energy - (free_atoms['O']+6*free_atoms['C']+8*free_atoms['H'])) )

Atomisation energy decomposition = -2.348972
Atomisation energy qm9 = -2.348972 


In [26]:
from ase.units import Bohr
nuclei[:,[1,2,3]]*Bohr

array([[ 7.7650813 ,  7.88070917, 11.9363884 ],
       [ 8.4602741 ,  8.06449135, 10.97157504],
       [ 9.74637634,  8.86001086, 10.96608324],
       [ 9.71319534,  9.99145101,  9.92604523],
       [11.06225644, 10.73239806,  9.82930211],
       [11.04400141, 11.80933083,  8.84754345],
       [11.00039509, 12.69065748,  8.031585  ],
       [ 8.18181841,  7.6425646 ,  9.97650387],
       [ 9.92533402,  9.24148572, 11.97580151],
       [10.5613193 ,  8.16454448, 10.7148987 ],
       [ 9.4577488 ,  9.58680401,  8.93984095],
       [ 8.92713964, 10.70976832, 10.18199237],
       [11.32734511, 11.13790602, 10.81432368],
       [11.85339595, 10.01501209,  9.57405719],
       [10.97431724, 13.47286448,  7.31406039]])

In [32]:
import numpy as np
np.array([nuclei[:,0],ae,at]).T

array([[  8.        , -52.41941963,  23.53488468],
       [  6.        , -38.81593713,  -0.07943982],
       [  6.        , -39.49926896,  -0.76277164],
       [  6.        , -39.88880231,  -1.152305  ],
       [  6.        , -38.57914187,   0.15735545],
       [  6.        , -36.29823107,   2.43826624],
       [  6.        , -32.4020716 ,   6.33442572],
       [  1.        ,  -5.39594217,  -4.00594386],
       [  1.        ,  -5.70814393,  -4.31814562],
       [  1.        ,  -5.64600143,  -4.25600311],
       [  1.        ,  -5.78079931,  -4.39080099],
       [  1.        ,  -5.79580329,  -4.40580498],
       [  1.        ,  -5.5379296 ,  -4.14793128],
       [  1.        ,  -5.53118386,  -4.14118554],
       [  1.        ,  -4.54357057,  -3.15357225]])